<a href="https://colab.research.google.com/github/Shape95/Wanted_Pre_Onboarding/blob/main/%EC%9B%90%ED%8B%B0%EB%93%9C_%ED%94%84%EB%A6%AC%EC%98%A8%EB%B3%B4%EB%94%A9_AI_ML_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [165]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
    
  def preprocessing(self, sequences):
    result = []
    # 조건 1 : 소문자 변환
    sequences = [a.lower() for a in sequences]
    # 조건 1 & 2 : 특수문자 제거 & 토큰화 
    for seq in sequences:
      lst=[]
      for word in seq.split(' '):
        tmp = ''.join(a for a in word if a.isalnum())
        lst.append(tmp)
      result.append(lst)    
    return result

  def fit(self, sequences):
    self.fit_checker = False
    # 조건 1 : 토큰화 수행
    tokens = self.preprocessing(sequences)
    # 조건 2 : word_dict 생성
    tokens_set = list(set(word for token in tokens for word in token))
    for i,k in enumerate(tokens_set):
      self.word_dict[k] = i+1
    self.fit_checker = True

  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      for token in tokens:
        for k, word in enumerate(token):
          if word in self.word_dict:
            token[k] = self.word_dict[word]
          else:
            token[k] = self.word_dict['oov']
      result = tokens
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [170]:
# 문제 1 테스트
token = Tokenizer()
lst2 = ['I go to school.', 'I LIKE pizza!']
lst = ['먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요']

# 1-1 (전처리)
print('preprocessing : {}'.format(token.preprocessing(lst)))

# 1-2 (word_dict 생성)
token.fit(lst)
print('fit : {}'.format(token.word_dict))

# 1-3 (정수 인덱싱)
print('transform : {}'.format(token.transform(lst)))

preprocessing : [['먹고', '싶은', '사과'], ['먹고', '싶은', '바나나'], ['길고', '노란', '바나나', '바나나'], ['저는', '과일이', '좋아요']]
fit : {'oov': 0, '사과': 1, '바나나': 2, '노란': 3, '싶은': 4, '길고': 5, '먹고': 6, '과일이': 7, '저는': 8, '좋아요': 9}
transform : [[6, 4, 1], [6, 4, 2], [5, 3, 2, 2], [8, 7, 9]]


### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요

In [171]:
from math import log
import pandas as pd

In [172]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
    
    # {'oov': 0.6931471805599453, 'i': -0.40546510810816444, 'like': 0.0, 'school': 0.0, 'go': 0.0, 'to': 0.0, 'pizza': 0.0}
    self.idf_dict = {}
    
    # {0: 'oov', 1: 'i', 2: 'like', 3: 'school', 4: 'go', 5: 'to', 6: 'pizza'}
    self.word_dict_inverse = {}

  # idf 생성식
  def idf(self, n, df):
    return log(n/(1+df))

  def fit(self, sequences):
    # tokenized, ex) [[3, 5, 2, 4], [3, 1, 6]]
    tokenized = self.tokenizer.fit_transform(sequences)
    self.word_dict_inverse = dict(zip(self.tokenizer.word_dict.values(), self.tokenizer.word_dict.keys()))
    n = len(tokenized)
    for i in range(len(self.word_dict_inverse)):
      df = 0
      for token in tokenized:
        if i in token:
          df+=1
      self.idf_dict[self.word_dict_inverse[i]] = self.idf(n, df)
    self.fit_checker = True

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      tfidf_matrix = []
      for token in tokenized:
        tfidf_matrix.append([])
        for i in range(len(self.word_dict_inverse)):
          tf = token.count(i)
          idf = self.idf_dict[self.word_dict_inverse[i]]
          tfidf_matrix[-1].append(tf*idf)
      return tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")
  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

In [173]:
# 문제 2 테스트
token = Tokenizer()
tfid = TfidfVectorizer(token)

# 2-1 (IDF 생성)
tfid.fit(lst)
idf = pd.DataFrame(tfid.idf_dict.values(), index = tfid.tokenizer.word_dict, columns=['IDF'])
print('Test 2-1 ----------------------------')
print(idf)
print()

# 2-2 (TF-IDF 생성)
print('Test 2-2 ----------------------------')
tfidf = tfid.transform(lst)
tfidf = pd.DataFrame(tfidf, columns=token.word_dict, index=lst)
tfidf



Test 2-1 ----------------------------
          IDF
oov  1.386294
사과   0.693147
바나나  0.287682
노란   0.693147
싶은   0.287682
길고   0.693147
먹고   0.287682
과일이  0.693147
저는   0.693147
좋아요  0.693147

Test 2-2 ----------------------------


,oov,사과,바나나,노란,싶은,길고,먹고,과일이,저는,좋아요
먹고 싶은 사과,0.0,0.693147,0.000000,0.000000,0.287682,0.000000,0.287682,0.000000,0.000000,0.000000
먹고 싶은 바나나,0.0,0.000000,0.287682,0.000000,0.287682,0.000000,0.287682,0.000000,0.000000,0.000000
길고 노란 바나나 바나나,0.0,0.000000,0.575364,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000
저는 과일이 좋아요,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147
